In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers.initializers import variance_scaling_initializer

import skimage.io as io
import numpy as np
from src.util import image as img_util
from matplotlib import pyplot as plt

import deepdish as dd
from os.path import join, dirname

from utilsTest import *

import os.path as osp
import os
import sys
curr_path = osp.dirname(os.getcwd())
model_dir = osp.join(curr_path, '..', 'models')

smpl_model_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\models\neutral_smpl_with_cocoplus_reg.pkl'
smpl_face_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\src\tf_smpl\smpl_faces.npy'

from src.tf_smpl.batch_lbs import batch_rodrigues
from src.tf_smpl.batch_smpl import SMPL
from src.tf_smpl.projection import batch_orth_proj_idrot

In [2]:
img_path = 'data/im1963.jpg'
json_path = None
input_img, proc_param, img = preprocess_image(img_path, json_path)
input_img = np.expand_dims(input_img, 0)
input_img = np.float32(input_img)

Resizing so the max image size is 224..


In [3]:
def load_mean_param():
    mean = np.zeros((1, 85))
    # Initialize scale at 0.9
    mean[0, 0] = 0.9
    mean_path = join(
        dirname(smpl_model_path), 'neutral_smpl_mean_params.h5')
    mean_vals = dd.io.load(mean_path)

    mean_pose = mean_vals['pose']
    # Ignore the global rotation.
    mean_pose[:3] = 0.
    mean_shape = mean_vals['shape']

    # This initializes the global pose to be up-right when projected
    mean_pose[0] = np.pi

    mean[0, 3:] = np.hstack((mean_pose, mean_shape))
    mean = tf.constant(mean, tf.float32)
    mean_var = tf.Variable(
        mean, name="mean_param", dtype=tf.float32, trainable=True)
    E_var.append(mean_var)
    init_mean = tf.tile(mean_var, [1, 1])
    return init_mean

In [4]:
use_3d_label = True
num_stage = 3
total_params = 85
num_cam = 3
num_theta = 72
batch_size = 1

In [5]:
img_enc_fn, threed_enc_fn = get_encoder_fn_separate('resnet_fc3_dropout')
img_feat, E_var = img_enc_fn(input_img, is_training=False, reuse=False)
smpl = SMPL(smpl_model_path)

loss_kps = []
if use_3d_label:
    loss_3d_joints, loss_3d_params = [], []
# For discriminator
fake_rotations, fake_shapes = [], []
# Start loop
# 85D
theta_prev = load_mean_param()

# For visualizations
all_verts = []
all_pred_kps = []
all_pred_cams = []
all_delta_thetas = []
all_theta_prev = []

# Main IEF loop
for i in np.arange(num_stage):
    print('Iteration %d' % i)
    # ---- Compute outputs
    state = tf.concat([img_feat, theta_prev], 1)

    if i == 0:
        delta_theta, _ = threed_enc_fn(state, num_output= total_params, reuse=False)
    else:
        delta_theta, _ = threed_enc_fn(state, num_output= total_params, reuse=True)

    # Compute new theta
    theta_here = theta_prev + delta_theta
    # cam = N x 3, pose N x self.num_theta, shape: N x 10
    cams = theta_here[:, :num_cam]
    poses = theta_here[:, num_cam:(num_cam + num_theta)]
    shapes = theta_here[:, (num_cam + num_theta):]
    # Rs_wglobal is Nx24x3x3 rotation matrices of poses
    verts, Js, pred_Rs = smpl(shapes, poses, get_skin=True)
    pred_kp = batch_orth_proj_idrot(Js, cams, name='proj2d_stage%d' % i)
    # --- Compute losses:
#     loss_kps.append(keypoint_loss(
#         self.kp_loader, pred_kp))
    pred_Rs = tf.reshape(pred_Rs, [-1, 24, 9])
#     if self.use_3d_label:
#         loss_poseshape, loss_joints = self.get_3d_loss(
#             pred_Rs, shapes, Js)
#         loss_3d_params.append(loss_poseshape)
#         loss_3d_joints.append(loss_joints)

    # Save pred_rotations for Discriminator
    fake_rotations.append(pred_Rs[:, 1:, :])
    fake_shapes.append(shapes)

    # Finally update to end iteration.
    theta_prev = theta_here

W0227 23:34:23.694994  1456 deprecation_wrapper.py:119] From C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\utilsTest.py:61: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



Iteration 0


W0227 23:34:24.009151  1456 deprecation.py:323] From C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\src\tf_smpl\batch_lbs.py:55: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Iteration 1
Reuse is on!
Iteration 2
Reuse is on!


In [6]:
fake_shapes

[<tf.Tensor 'strided_slice_2:0' shape=(1, 10) dtype=float32>,
 <tf.Tensor 'strided_slice_6:0' shape=(1, 10) dtype=float32>,
 <tf.Tensor 'strided_slice_10:0' shape=(1, 10) dtype=float32>]

In [7]:
fake_rotations

[<tf.Tensor 'strided_slice_3:0' shape=(1, 23, 9) dtype=float32>,
 <tf.Tensor 'strided_slice_7:0' shape=(1, 23, 9) dtype=float32>,
 <tf.Tensor 'strided_slice_11:0' shape=(1, 23, 9) dtype=float32>]

In [8]:
all_fake_rotations = tf.reshape(
    tf.concat(fake_rotations, 0),
    [batch_size * num_stage, -1, 9])
all_fake_rotations

<tf.Tensor 'Reshape_3:0' shape=(3, 23, 9) dtype=float32>

In [9]:
all_fake_shapes = tf.concat(fake_shapes, 0)
all_fake_shapes

<tf.Tensor 'concat_4:0' shape=(3, 10) dtype=float32>

In [10]:
def Discriminator_separable_rotations(
        poses,
        shapes,
        weight_decay,
):
    """
    23 Discriminators on each joint + 1 for all joints + 1 for shape.
    To share the params on rotations, this treats the 23 rotation matrices
    as a "vertical image":
    Do 1x1 conv, then send off to 23 independent classifiers.

    Input:
    - poses: N x 23 x 1 x 9, NHWC ALWAYS!!
    - shapes: N x 10
    - weight_decay: float

    Outputs:
    - prediction: N x (1+23) or N x (1+23+1) if do_joint is on.
    - variables: tf variables
    """
    data_format = "NHWC"
    with tf.name_scope("Discriminator_sep_rotations", [poses, shapes]):
        with tf.variable_scope("D") as scope:
            with slim.arg_scope(
                [slim.conv2d, slim.fully_connected],
                    weights_regularizer=slim.l2_regularizer(weight_decay)):
                with slim.arg_scope([slim.conv2d], data_format=data_format):
                    poses = slim.conv2d(poses, 32, [1, 1], scope='D_conv1')
                    poses = slim.conv2d(poses, 32, [1, 1], scope='D_conv2')
                    theta_out = []
                    for i in range(0, 23):
                        theta_out.append(
                            slim.fully_connected(
                                poses[:, i, :, :],
                                1,
                                activation_fn=None,
                                scope="pose_out_j%d" % i))
                    theta_out_all = tf.squeeze(tf.stack(theta_out, axis=1))

                    # Do shape on it's own:
                    shapes = slim.stack(
                        shapes,
                        slim.fully_connected, [10, 5],
                        scope="shape_fc1")
                    shape_out = slim.fully_connected(
                        shapes, 1, activation_fn=None, scope="shape_final")
                    
                    """ Compute joint correlation prior!"""
                    nz_feat = 1024
                    poses_all = slim.flatten(poses, scope='vectorize')
                    poses_all = slim.fully_connected(
                        poses_all, nz_feat, scope="D_alljoints_fc1")
                    poses_all = slim.fully_connected(
                        poses_all, nz_feat, scope="D_alljoints_fc2")
                    poses_all_out = slim.fully_connected(
                        poses_all,
                        1,
                        activation_fn=None,
                        scope="D_alljoints_out")
                    out = tf.concat([theta_out_all,
                                     poses_all_out, shape_out], 1)

            variables = tf.contrib.framework.get_variables(scope)
            return out, variables

In [11]:
def batch_rodrigues(theta, name=None):
    """
    Theta is N x 3
    """
    # with tf.name_scope(name, "batch_rodrigues", [theta]):
    with tf.name_scope(name, "batch_rodrigues"):
        batch_size = theta.shape.as_list()[0]
        
        angle = tf.expand_dims(tf.norm(theta + 1e-8, axis=1), -1)
        r = tf.expand_dims(tf.div(theta, angle), -1)

        angle = tf.expand_dims(angle, -1)
        cos = tf.cos(angle)
        sin = tf.sin(angle)

        outer = tf.matmul(r, r, transpose_b=True, name="outer")

        eyes = tf.tile(tf.expand_dims(tf.eye(3), 0), [batch_size, 1, 1])
        R = cos * eyes + (1 - cos) * outer + sin * batch_skew(
            r, batch_size=batch_size)
        return R

In [14]:
def setup_discriminator(fake_rotations, fake_shapes):
    # Compute the rotation matrices of "rea" pose.
    # These guys are in 24 x 3.
    real_rotations = batch_rodrigues(tf.reshape(pose_loader, [-1, 3]))
    real_rotations = tf.reshape(real_rotations, [-1, 24, 9])
    # Ignoring global rotation. N x 23*9
    # The # of real rotation is B*num_stage so it's balanced.
    real_rotations = real_rotations[:, 1:, :]
    all_fake_rotations = tf.reshape(
        tf.concat(fake_rotations, 0),
        [batch_size * num_stage, -1, 9])
    comb_rotations = tf.concat(
        [real_rotations, all_fake_rotations], 0, name="combined_pose")
    comb_rotations = tf.expand_dims(comb_rotations, 2)

    all_fake_shapes = tf.concat(fake_shapes, 0)
    comb_shapes = tf.concat(
        [shape_loader, all_fake_shapes], 0, name="combined_shape")

    disc_input = {
        'weight_decay': d_wd,
        'shapes': comb_shapes,
        'poses': comb_rotations
    }

    d_out, D_var = Discriminator_separable_rotations(
        **disc_input)

    d_out_real, d_out_fake = tf.split(d_out, 2)
    # Compute losses:
    with tf.name_scope("comp_d_loss"):
        d_loss_real = tf.reduce_mean(
            tf.reduce_sum((d_out_real - 1)**2, axis=1))
        d_loss_fake = tf.reduce_mean(
            tf.reduce_sum((d_out_fake)**2, axis=1))
        # Encoder loss
        e_loss_disc = tf.reduce_mean(
            tf.reduce_sum((d_out_fake - 1)**2, axis=1))